# Design considerations:
    
Setting up the Kitty:
    1. Create event for the kitty
    2. Create a member
    3. Add all of the relevant members to the event
    4. Create a transaction (associated with a member already created)
        -We ensure the amount entered is a number
    5. Add the transaction to the kitty
       - We ensure the member associated with the transaction is in the kitty event, if not we print out a relevant message
       
       
       
 I've also written the code in a way that a user can at all times check relevant information pertaining to events such as:
 1. What the total spend of the kitty is
 2. Which members have spent the most (lenders)
 3. Which members owe more than they have contributed (borrowers)
 4. The members in the kitty
 5. The list of transactions made to date



In [ ]:
#Member class allows us to create a member
        
        
class member:
    
    #Create class variable member. Value increases by one, each time a member is instantiated
    member_id=1
    
    def __init__(self,person_name):
        self.person_name=person_name
        self.member_id = member.member_id
        #Update member_id after each instantiation
        member.member_id+=1
    
        

In [ ]:
#Class to create a transaction, pass in member object which will contain member id/info
class transaction:
    
    
    #create a transaction, pass in an instance of member. Make sure a digit is entered in the amount_paid
    def __init__(self, activity_name, amount_paid, member):
        self.activity_name = activity_name
        self.member = member
        if type(amount_paid) == int or type(amount_paid)==float:
            self.amount_paid = amount_paid
        else:
            print("Please enter a valid amount in your transaction.")

In [ ]:
# Create the Kitty class
# A Kitty class will store members of the kitty, store the transactions for the kitty and calculate at the end how 
#much each person owes
class kitty:
    

    #Init method
    def __init__(self,event_name):
        self.event_name = event_name
        self.total_spent = 0
        self.count_members = 0
        self.count_trans = 0
        self.kitty_members = []
        self.transactionslist = []
        self.settled = []
        self.borrowers = []
        self.givers = []
        self.spenders = {}
        self.total_owedPer = 0
        
    #Add a transaction to the specific event instance of the class kitty, keeping a tally of total spend and the count of transactions
    def add_transaction(self, transaction):
        amount_paid = transaction.amount_paid
        
        #If the member is not in the kitty for a specific event, print out an error message   
        if transaction.member not in self.kitty_members:
            print("This person did not participate in the event.Transaction not added.")
           
                
        #Else, if not already,  add the transaction to the transactions list. 
        #Add transaction amount to total tally. Update amound owed each.
   
        else:
            if transaction not in self.transactionslist:
                self.transactionslist.append(transaction)
                self.count_trans+=1
                self.total_spent+=amount_paid
                self.amount_owedEach()
              
            # If this is the first time a member is adding a transaction then insert the member in spenders
            # If the member is already in spenders, then update the amount the member has already spent.
            if transaction.member in self.spenders:
                print("Updating member transaction")
                newAmount = self.spenders[transaction.member] + transaction.amount_paid
                print("MemberId:",transaction.member.member_id ," has now contributed: €", newAmount)
                self.spenders[transaction.member] = newAmount
            else:
                print("Adding new member transaction")
                self.spenders[transaction.member] = transaction.amount_paid

            
        
    #Display transactions in the instance of the kitty
    def display_transaction(self):
        for transaction in self.transactionslist:
            print("MemberId:", transaction.member.member_id, "paid €:", transaction.amount_paid)
            
               
    #Add a member to the event instance of the kitty
    def add_member(self, member):
        if member not in self.kitty_members:
            self.kitty_members.append(member)
            self.count_members+=1
            self.spenders[member] = 0
            self.amount_owedEach()
        else:
            print("Member is already a member of the kitty.")
            
        
        
    #Amount owed each (before settling etc)
    def amount_owedEach(self):
        #Avoid a /0 error
        if self.count_members == 0:
            print("No members yet added to the kitty to calculate the amount owed per person.")
            self.total_owedPer=0
            return self.total_owedPer
            
        else:
            self.total_owedPer = self.total_spent/self.count_members
            return self.total_owedPer
            
    
    
    

#     Settle expenses
#     Create three lists "Lenders","Borrowers","Settlers"
#     Remove total_owedPer from each person (as everyone will need to contribute X amount when all settled)
#     Find max(lender), max(borrower) and settle this way

    
    def balancePerPerson(self):
        transactions_lessOwed = []
        settled = self.settled
        borrowers = self.borrowers
        givers = self.givers
        
        
        #Print total spend. That is X each.
        print("Total: €",self.total_spent,"that is","€",round((self.total_owedPer),2),"each.")
        
        
        
        for transaction in self.transactionslist:
            transactions_lessOwed.append(transaction)

        #Remove amount owed from everyone's balance = add to new list transactions_lessOwed
        for key, value in self.spenders.items():
            self.spenders[key] = value-self.total_owedPer
    
        #If anyone has a balance of 0, add them to "settled" list
        for key, value in self.spenders.items():
            if value == 0:
                settled.append([key, value])
    
        #If they borrowed more than they owe, add them to borrower list
            elif value < 0:
                borrowers.append([key, value])
        #If they gave more than they owe, add them to a givers list
            elif value > 0:
                givers.append([key, value])
        
           
            
         #Print X has balance of Y 
        for key, value in self.spenders.items():
            print(key.person_name,"has balance €", round(value,2))

          
        print("!!------------------------------------------!!")
        print("         Start simplification of Debt         ")
        print("!!------------------------------------------!!")

        
        #While all members are not in settled:
        #-Find max lender, and biggest borrower(read: MIN borrower as it is a negative value)
        while len(settled)+1 < self.count_members:
            # This object will represents an object of [memberObject, amountOwed]
            max_giver = [0, 0]
            index_of_max_giver = 0
            count_index = 0
            for giver in givers:
                if giver[1] > max_giver[1]:
                    max_giver = giver
                    index_of_max_giver = count_index
                count_index += 1
                
            
            count_index = 0 
            index_of_max_borrower = 0
            
            # This object will represents an object of [memberObject, amountPaid]
            max_borrower = [0, 0]
            for borrower in borrowers:
                if borrower[1] < max_borrower[1]:
                    max_borrower = borrower
                    index_of_max_borrower = count_index
                count_index += 1
            
            
            if (max_giver[1] < abs(max_borrower[1])):
                print(str(max_borrower[0].person_name) + " pays: €" +str(abs(round(max_giver[1],2)))+ " to "+ str(max_giver[0].person_name))
                # giver is settled
                settled.append(max_giver)
                del givers[index_of_max_giver]
                borrowers[index_of_max_borrower][1] += max_giver[1]
                
            elif (max_giver[1] > abs(max_borrower[1])):
                print(str(max_borrower[0].person_name) + " pays: €" +str(abs(round(max_borrower[1],2)))+ " to "+ str(max_giver[0].person_name))
                # borrower is settled
                settled.append(max_borrower)
                # remove borrower from borrowers list
                del borrowers[index_of_max_borrower]
                givers[index_of_max_giver][1] += max_borrower[1]
                # update lender amount owed
                
            else:
                print(str(max_borrower[0].person_name) + " pays: €" +str(abs(round(max_borrower[1],2)))+ " to "+ str(max_giver[0].person_name))
                # borrower and giver settle
                settled.append(max_borrower)
                settled.append(max_giver)
                del borrowers[index_of_max_borrower]
                del givers[index_of_max_giver]
                
                
         
            
            
            
            
            
            

        
         
            

# Sample 1: Creation

In [ ]:
concert = kitty("concert")
concert.kitty_members


#CREATE MEMBERS

#CREATE A MEMBER 1
annie=member("Annie")

#CREATE A MEMBER 2
sally=member("Sally")

#CREATE A MEMBER 3
bill=member("Bill")




#ADD THE MEMBERS TO THE KITTY EVENT


concert.add_member(annie)
concert.add_member(sally)
concert.add_member(bill)




#CREATE TRANSACTIONS


tickets=transaction("tickets", 180, annie)
dinner=transaction("dinner", 75, sally)
drinks=transaction("drinks", 19, bill)
taxi=transaction("taxi", 16, bill)


#ADD ABOVE TRANSACTIONS TO KITTY FOR THE CONCERT


concert.add_transaction(tickets)
concert.add_transaction(dinner)
concert.add_transaction(drinks)
concert.add_transaction(taxi)




# Sample 1: Output

In [ ]:
#ANSWER OUTPUT:

concert.balancePerPerson()

# Sample 2: Creation

In [ ]:
cinema = kitty("cinema")



#CREATE MEMBERS
cathy=member("Cathy")
robin=member("Robin")
jen=member("Jen")



#ADD MEMBERS TO THE EVENT KITTY
cinema.add_member(cathy)
cinema.add_member(robin)
cinema.add_member(jen)



#CREATE TRANSACTIONS
tickets=transaction("tickets", 33, cathy)
dinner=transaction("dinner", 60, robin)
drinks=transaction("drinks", 21, jen)
taxi=transaction("taxi", 27, jen)


#ADD ABOVE TRANSACTIONS TO KITTY FOR THE CONCERT

cinema.add_transaction(tickets)
cinema.add_transaction(dinner)
cinema.add_transaction(drinks)
cinema.add_transaction(taxi)



# Sample 2: output

In [ ]:
#ANSWER OUTPUT:

cinema.balancePerPerson()


# Sample 3: creation

In [ ]:
#ANSWER OUTPUT:
weekendAway = kitty("weekend")


#CREATE MEMBERS
nora=member("Nora")
eva=member("Eva")
frankie=member("Frankie")
harry=member("Harry")


#ADD MEMBERS TO THE EVENT KITTY
weekendAway.add_member(nora)
weekendAway.add_member(eva)
weekendAway.add_member(frankie)
weekendAway.add_member(harry)


#CREATE TRANSACTIONS
dinner_Fri=transaction("dinner_Fri", 110, nora)
lunch_Sat=transaction("lunch_Sat", 60, eva)
dinner_Sat=transaction("dinner_Sat", 125, frankie)
harry_Sun=transaction("lunch_Sun", 70, harry)


#ADD ABOVE TRANSACTIONS TO KITTY FOR THE CONCERT

weekendAway.add_transaction(dinner_Fri)
weekendAway.add_transaction(lunch_Sat)
weekendAway.add_transaction(dinner_Sat)
weekendAway.add_transaction(harry_Sun)


# Sample 3: output

In [ ]:
#ANSWER OUTPUT:
#Note, Harry pays two transactions to Frankie in my output while Eva pays two transactions
#In the output given in the assignment question

#Here, Eva pays her entire balance to Frankie 
#Harry splits his payment between Nora and Frankie to make the total amount

weekendAway.balancePerPerson()

